### Performance of IPS's Ray-Tracing

Standard Settings are:
- test_amount=50
- step_size=10
- reflexion_order=3
- ray_scaling=True
- detail_draw=False
- output_format="channels"

Investigated Factors:
- `Ray Amount` (step_size)
- `Scaling of Rays` (ray_scaling)
- `Reflexion Order` (reflexion_order)
- `Detail Drawing of Rays` (detail_draw)

Helper:

In [1]:
import sys
sys.path += [".."] # os.path.abspath("../img-phy-sim")

import img_phy_sim as ips
from physgen_dataset import get_image  # local module

import time

import numpy as np

d:\Programme\Anaconda\envs\img-phy-sim\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [8]:
def get_random_image(name=None):
    # get image
    input_ = get_image(mode='train', variation="sound_reflection", input_type="osm", output_type="standard", 
                shuffle=True, return_output=False, as_numpy_array=True)

    if name is None:
        input_src = "./cache_img_1_input.png"
    else:
        input_src = name
    ips.img.save(input_, input_src, should_scale=False)

    return input_src

def ray_performance_test(imgs, step_size, reflexion_order, ray_scaling, detail_draw, output_format):
    test_amount = len(imgs)

    ray_compute_times = []
    ray_draw_times = []
    times = []

    for input_src in imgs:
        # input_src = get_random_image()
        input_ = ips.img.open(src=input_src, should_scale=False, should_print=False)
        
        start = time.time()
        # calc rays
        rays = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                        img_src=input_src, 
                                        directions_in_degree=ips.ray_tracing.get_linear_degree_range(step_size=step_size),
                                        wall_values=None, 
                                        wall_thickness=0,
                                        img_border_also_collide=False,
                                        reflexion_order=reflexion_order,
                                        should_scale_rays=ray_scaling,
                                        should_scale_img=False)
        ray_compute_time = time.time() - start

        # show rays on input
        start = time.time()
        ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=detail_draw, 
                                            output_format=output_format, 
                                            img_background=input_, ray_value=2, ray_thickness=1, 
                                            img_shape=(256, 256), dtype=float, standard_value=0,
                                            should_scale_rays_to_image=ray_scaling, original_max_width=None, original_max_height=None,
                                            show_only_reflections=True)
        ray_draw_time = time.time() - start

        ray_compute_times += [ray_compute_time]
        ray_draw_times += [ray_draw_time]
        times += [ray_draw_time+ray_compute_time]

    avg_time = np.mean(times)
    median_time = np.median(times)
    var_time = np.var(times)

    avg_compute_time = np.mean(ray_compute_times)
    median_compute_time = np.median(ray_compute_times)
    var_compute_time = np.var(ray_compute_times)

    avg_draw_time = np.mean(ray_draw_times)
    median_draw_time = np.median(ray_draw_times)
    var_draw_time = np.var(ray_draw_times)

    print_str = f"\n{'-'*2}\nPerformance Result with {test_amount} Tests\n"
    print_str += f"Total Time:\n    Mean = {avg_time}\n    Median = {median_time}\n    Variance = {var_time}"
    print_str += f"\n\nRay Compute Time:\n    Mean = {avg_compute_time}\n    Median = {median_compute_time}\n    Variance = {var_compute_time}"
    print_str += f"\n\nRay Draw Time:\n    Mean = {avg_draw_time}\n    Median = {median_draw_time}\n    Variance = {var_draw_time}"

    print(print_str)

    return [
        avg_time,
        median_time,
        var_time,
        avg_compute_time,
        median_compute_time,
        var_compute_time,
        avg_draw_time,
        median_draw_time,
        var_draw_time,
    ]

def compare_results(results):
    results = np.array(results)
    num_experiments = len(results)

    metrics = [
        "avg_time", "median_time", "var_time",
        "avg_compute_time", "median_compute_time", "var_compute_time",
        "avg_draw_time", "median_draw_time", "var_draw_time"
    ]

    summary = f"\nNumber of experiments: {num_experiments}\n"

    # Analyze each metric
    for i, name in enumerate(metrics):
        values = results[:, i]

        mean_val = np.mean(values)
        std_val = np.std(values)
        min_val = np.min(values)
        max_val = np.max(values)
        rel_change = (max_val - min_val) / mean_val * 100 if mean_val != 0 else 0

        summary += f"\n\
            {name:20s}: mean={mean_val:.4f}, std={std_val:.4f}, min={min_val:.4f}, max={max_val:.4f}, rel_change={rel_change:.2f}%"

    # Check overall trend in avg_time
    avg_times = results[:, 0]
    trend = np.polyfit(np.arange(num_experiments), avg_times, 1)[0]
    if trend > 0:
        trend_desc = "increasing"
    elif trend < 0:
        trend_desc = "decreasing"
    else:
        trend_desc = "stable"

    summary += f"\n\nOverall trend in avg_time: {trend_desc} ({trend:.4e} change per experiment)"
    summary += f"\nConclusion: Performance changes {('significantly' if np.std(avg_times) / np.mean(avg_times) > 0.1 else 'slightly')} across experiments."

    print(summary)

    return summary

In [3]:
test_amount = 50
imgs = []
for i in range(test_amount):
    name = f"./cache_img_{i+1}.png"
    get_random_image(name=name)
    imgs += [name]

PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Dataset for train got created
PhysGen (sound_reflection) Data

**Experiment 1: Ray Amount**

In [9]:
results = []
results += [ray_performance_test(imgs=imgs, 
                                step_size=10, 
                                reflexion_order=3, 
                                ray_scaling=True, 
                                detail_draw=False, 
                                output_format="channels")]

d:\Programme\Anaconda\envs\img-phy-sim\Lib\site-packages\numpy\_core\numeric.py:353: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(a, fill_value, casting='unsafe')



--
Performance Result with 50 Tests
Total Time:
    Mean = 0.5743922901153564
    Median = 0.5546430349349976
    Variance = 0.007568072940822822

Ray Compute Time:
    Mean = 0.5529049491882324
    Median = 0.5345596075057983
    Variance = 0.007408498095403866

Ray Draw Time:
    Mean = 0.021487340927124024
    Median = 0.020676851272583008
    Variance = 3.813566111512045e-06


In [10]:
results += [ray_performance_test(imgs=imgs, 
                                step_size=5, 
                                reflexion_order=3, 
                                ray_scaling=True, 
                                detail_draw=False, 
                                output_format="channels")]


--
Performance Result with 50 Tests
Total Time:
    Mean = 1.1274364471435547
    Median = 1.103517770767212
    Variance = 0.02612410859151851

Ray Compute Time:
    Mean = 1.080513834953308
    Median = 1.0601667165756226
    Variance = 0.024990209654716294

Ray Draw Time:
    Mean = 0.046922612190246585
    Median = 0.04627180099487305
    Variance = 3.059765149885152e-05


In [11]:
results += [ray_performance_test(imgs=imgs, 
                                step_size=1, 
                                reflexion_order=3, 
                                ray_scaling=True, 
                                detail_draw=False, 
                                output_format="channels")]


--
Performance Result with 50 Tests
Total Time:
    Mean = 5.531743464469909
    Median = 5.293457984924316
    Variance = 0.6109478504450768

Ray Compute Time:
    Mean = 5.312908511161805
    Median = 5.0782389640808105
    Variance = 0.580682634727182

Ray Draw Time:
    Mean = 0.21883495330810546
    Median = 0.21196365356445312
    Variance = 0.0004697135053844249


In [13]:
results += [ray_performance_test(imgs=imgs, 
                                step_size=0.1, 
                                reflexion_order=3, 
                                ray_scaling=True, 
                                detail_draw=False, 
                                output_format="channels")]


--
Performance Result with 50 Tests
Total Time:
    Mean = 56.791828088760376
    Median = 54.779919385910034
    Variance = 67.8258524735977

Ray Compute Time:
    Mean = 53.481501364707945
    Median = 51.57227301597595
    Variance = 58.48254623186749

Ray Draw Time:
    Mean = 3.310326724052429
    Median = 3.07047700881958
    Variance = 0.6096038808240902


In [14]:
compare_results(results);


Number of experiments: 4

            avg_time            : mean=16.0064, std=23.6257, min=0.5744, max=56.7918, rel_change=351.22%
            median_time         : mean=15.4329, std=22.7908, min=0.5546, max=54.7799, rel_change=351.36%
            var_time            : mean=17.1176, std=29.2774, min=0.0076, max=67.8259, rel_change=396.19%
            avg_compute_time    : mean=15.1070, std=22.2322, min=0.5529, max=53.4815, rel_change=350.36%
            median_compute_time : mean=14.5613, std=21.4404, min=0.5346, max=51.5723, rel_change=350.50%
            var_compute_time    : mean=14.7739, std=25.2362, min=0.0074, max=58.4825, rel_change=395.80%
            avg_draw_time       : mean=0.8994, std=1.3940, min=0.0215, max=3.3103, rel_change=365.67%
            median_draw_time    : mean=0.8373, std=1.2914, min=0.0207, max=3.0705, rel_change=364.22%
            var_draw_time       : mean=0.1525, std=0.2639, min=0.0000, max=0.6096, rel_change=399.67%

Overall trend in avg_time: increasin

**Experiment 2: Ray Scaling**

In [15]:
results = []
results += [ray_performance_test(imgs=imgs, 
                                step_size=10, 
                                reflexion_order=3, 
                                ray_scaling=True, 
                                detail_draw=False, 
                                output_format="channels")]


--
Performance Result with 50 Tests
Total Time:
    Mean = 0.5822781038284301
    Median = 0.5637214183807373
    Variance = 0.007754208688502081

Ray Compute Time:
    Mean = 0.5605262660980225
    Median = 0.5423493385314941
    Variance = 0.00754003197469765

Ray Draw Time:
    Mean = 0.021751837730407717
    Median = 0.02132093906402588
    Variance = 4.26059041658391e-06


In [16]:
results += [ray_performance_test(imgs=imgs, 
                                step_size=10, 
                                reflexion_order=3, 
                                ray_scaling=False, 
                                detail_draw=False, 
                                output_format="channels")]


--
Performance Result with 50 Tests
Total Time:
    Mean = 0.5665883159637451
    Median = 0.5478074550628662
    Variance = 0.007766807581710417

Ray Compute Time:
    Mean = 0.5557410860061646
    Median = 0.5372400283813477
    Variance = 0.007549793427198007

Ray Draw Time:
    Mean = 0.010847229957580567
    Median = 0.010637760162353516
    Variance = 2.102514868261096e-06


In [17]:
compare_results(results);


Number of experiments: 2

            avg_time            : mean=0.5744, std=0.0078, min=0.5666, max=0.5823, rel_change=2.73%
            median_time         : mean=0.5558, std=0.0080, min=0.5478, max=0.5637, rel_change=2.86%
            var_time            : mean=0.0078, std=0.0000, min=0.0078, max=0.0078, rel_change=0.16%
            avg_compute_time    : mean=0.5581, std=0.0024, min=0.5557, max=0.5605, rel_change=0.86%
            median_compute_time : mean=0.5398, std=0.0026, min=0.5372, max=0.5423, rel_change=0.95%
            var_compute_time    : mean=0.0075, std=0.0000, min=0.0075, max=0.0075, rel_change=0.13%
            avg_draw_time       : mean=0.0163, std=0.0055, min=0.0108, max=0.0218, rel_change=66.90%
            median_draw_time    : mean=0.0160, std=0.0053, min=0.0106, max=0.0213, rel_change=66.86%
            var_draw_time       : mean=0.0000, std=0.0000, min=0.0000, max=0.0000, rel_change=67.83%

Overall trend in avg_time: decreasing (-1.5690e-02 change per experim

**Experiment 3: Reflexion Order**

In [18]:
results = []
results += [ray_performance_test(imgs=imgs, 
                                step_size=10, 
                                reflexion_order=0, 
                                ray_scaling=True, 
                                detail_draw=False, 
                                output_format="channels")]


--
Performance Result with 50 Tests
Total Time:
    Mean = 0.3231235551834106
    Median = 0.32531797885894775
    Variance = 0.00024351806260958713

Ray Compute Time:
    Mean = 0.30882523536682127
    Median = 0.3108137845993042
    Variance = 0.00020978370210477808

Ray Draw Time:
    Mean = 0.014298319816589355
    Median = 0.014383316040039062
    Variance = 1.616508346273804e-06


In [19]:
results += [ray_performance_test(imgs=imgs, 
                                step_size=10, 
                                reflexion_order=1, 
                                ray_scaling=True, 
                                detail_draw=False, 
                                output_format="channels")]


--
Performance Result with 50 Tests
Total Time:
    Mean = 0.4432172107696533
    Median = 0.4405524730682373
    Variance = 0.0010647981035345766

Ray Compute Time:
    Mean = 0.42517398834228515
    Median = 0.423565149307251
    Variance = 0.0010190022018649872

Ray Draw Time:
    Mean = 0.018043222427368163
    Median = 0.01798272132873535
    Variance = 2.138532874050725e-06


In [20]:
results += [ray_performance_test(imgs=imgs, 
                                step_size=10, 
                                reflexion_order=2, 
                                ray_scaling=True, 
                                detail_draw=False, 
                                output_format="channels")]


--
Performance Result with 50 Tests
Total Time:
    Mean = 0.5088210201263428
    Median = 0.49715447425842285
    Variance = 0.0033666517140294675

Ray Compute Time:
    Mean = 0.4891412973403931
    Median = 0.47770631313323975
    Variance = 0.003280755523203993

Ray Draw Time:
    Mean = 0.01967972278594971
    Median = 0.019591569900512695
    Variance = 2.340457808281826e-06


In [21]:
results += [ray_performance_test(imgs=imgs, 
                                step_size=10, 
                                reflexion_order=4, 
                                ray_scaling=True, 
                                detail_draw=False, 
                                output_format="channels")]


--
Performance Result with 50 Tests
Total Time:
    Mean = 0.6324062347412109
    Median = 0.605924129486084
    Variance = 0.013463466656060064

Ray Compute Time:
    Mean = 0.6096883249282837
    Median = 0.5840120315551758
    Variance = 0.01301173935468953

Ray Draw Time:
    Mean = 0.022717909812927248
    Median = 0.02194070816040039
    Variance = 5.36553779213591e-06


In [22]:
results += [ray_performance_test(imgs=imgs, 
                                step_size=10, 
                                reflexion_order=16, 
                                ray_scaling=True, 
                                detail_draw=False, 
                                output_format="channels")]


--
Performance Result with 50 Tests
Total Time:
    Mean = 1.2316731643676757
    Median = 1.1406371593475342
    Variance = 0.22054101580450935

Ray Compute Time:
    Mean = 1.1960439109802246
    Median = 1.1085728406906128
    Variance = 0.21218104800083792

Ray Draw Time:
    Mean = 0.035629253387451175
    Median = 0.03327476978302002
    Variance = 8.594995888151969e-05


In [23]:
results += [ray_performance_test(imgs=imgs, 
                                step_size=10, 
                                reflexion_order=64, 
                                ray_scaling=True, 
                                detail_draw=False, 
                                output_format="channels")]


--
Performance Result with 50 Tests
Total Time:
    Mean = 2.440870976448059
    Median = 2.091689944267273
    Variance = 2.5420986134331596

Ray Compute Time:
    Mean = 2.3797806549072265
    Median = 2.040102481842041
    Variance = 2.439573844900538

Ray Draw Time:
    Mean = 0.06109032154083252
    Median = 0.05214428901672363
    Variance = 0.0010922406721758536


In [24]:
compare_results(results);


Number of experiments: 6

            avg_time            : mean=0.9300, std=0.7354, min=0.3231, max=2.4409, rel_change=227.71%
            median_time         : mean=0.8502, std=0.6128, min=0.3253, max=2.0917, rel_change=207.76%
            var_time            : mean=0.4635, std=0.9329, min=0.0002, max=2.5421, rel_change=548.45%
            avg_compute_time    : mean=0.9014, std=0.7194, min=0.3088, max=2.3798, rel_change=229.74%
            median_compute_time : mean=0.8241, std=0.6002, min=0.3108, max=2.0401, rel_change=209.83%
            var_compute_time    : mean=0.4449, std=0.8953, min=0.0002, max=2.4396, rel_change=548.32%
            avg_draw_time       : mean=0.0286, std=0.0160, min=0.0143, max=0.0611, rel_change=163.74%
            median_draw_time    : mean=0.0266, std=0.0129, min=0.0144, max=0.0521, rel_change=142.21%
            var_draw_time       : mean=0.0002, std=0.0004, min=0.0000, max=0.0011, rel_change=550.06%

Overall trend in avg_time: increasing (3.7365e-01 chan

**Experiment 4: Detail Draw**

In [25]:
results = []
results += [ray_performance_test(imgs=imgs, 
                                step_size=10, 
                                reflexion_order=3, 
                                ray_scaling=True, 
                                detail_draw=False, 
                                output_format="channels")]


--
Performance Result with 50 Tests
Total Time:
    Mean = 0.5766887855529785
    Median = 0.5593326091766357
    Variance = 0.007429155155891386

Ray Compute Time:
    Mean = 0.5552471351623535
    Median = 0.5382789373397827
    Variance = 0.007188621607549612

Ray Draw Time:
    Mean = 0.021441650390625
    Median = 0.02087223529815674
    Variance = 3.855979837226187e-06


In [26]:
results += [ray_performance_test(imgs=imgs, 
                                step_size=10, 
                                reflexion_order=3, 
                                ray_scaling=True, 
                                detail_draw=True, 
                                output_format="channels")]


--
Performance Result with 50 Tests
Total Time:
    Mean = 0.6796124744415283
    Median = 0.6607975959777832
    Variance = 0.015524363902080585

Ray Compute Time:
    Mean = 0.5591672611236572
    Median = 0.5477724075317383
    Variance = 0.007581451134702183

Ray Draw Time:
    Mean = 0.1204452133178711
    Median = 0.1117478609085083
    Variance = 0.0022347650876175973


In [27]:
compare_results(results);


Number of experiments: 2

            avg_time            : mean=0.6282, std=0.0515, min=0.5767, max=0.6796, rel_change=16.39%
            median_time         : mean=0.6101, std=0.0507, min=0.5593, max=0.6608, rel_change=16.63%
            var_time            : mean=0.0115, std=0.0040, min=0.0074, max=0.0155, rel_change=70.54%
            avg_compute_time    : mean=0.5572, std=0.0020, min=0.5552, max=0.5592, rel_change=0.70%
            median_compute_time : mean=0.5430, std=0.0047, min=0.5383, max=0.5478, rel_change=1.75%
            var_compute_time    : mean=0.0074, std=0.0002, min=0.0072, max=0.0076, rel_change=5.32%
            avg_draw_time       : mean=0.0709, std=0.0495, min=0.0214, max=0.1204, rel_change=139.55%
            median_draw_time    : mean=0.0663, std=0.0454, min=0.0209, max=0.1117, rel_change=137.05%
            var_draw_time       : mean=0.0011, std=0.0011, min=0.0000, max=0.0022, rel_change=199.31%

Overall trend in avg_time: increasing (1.0292e-01 change per ex

<br><br>

**All Results**

Executed with 50 random images.


<br><br>

Experiment 1: Ray Amount

```text
Number of experiments: 4

            avg_time            : mean=16.0064, std=23.6257, min=0.5744, max=56.7918, rel_change=351.22%
            median_time         : mean=15.4329, std=22.7908, min=0.5546, max=54.7799, rel_change=351.36%
            var_time            : mean=17.1176, std=29.2774, min=0.0076, max=67.8259, rel_change=396.19%
            avg_compute_time    : mean=15.1070, std=22.2322, min=0.5529, max=53.4815, rel_change=350.36%
            median_compute_time : mean=14.5613, std=21.4404, min=0.5346, max=51.5723, rel_change=350.50%
            var_compute_time    : mean=14.7739, std=25.2362, min=0.0074, max=58.4825, rel_change=395.80%
            avg_draw_time       : mean=0.8994, std=1.3940, min=0.0215, max=3.3103, rel_change=365.67%
            median_draw_time    : mean=0.8373, std=1.2914, min=0.0207, max=3.0705, rel_change=364.22%
            var_draw_time       : mean=0.1525, std=0.2639, min=0.0000, max=0.6096, rel_change=399.67%

Overall trend in avg_time: increasing (1.7306e+01 change per experiment)
Conclusion: Performance changes significantly across experiments.
```


<br><br>

Experiment 2: Ray Scaling

```text
Number of experiments: 2

            avg_time            : mean=0.5744, std=0.0078, min=0.5666, max=0.5823, rel_change=2.73%
            median_time         : mean=0.5558, std=0.0080, min=0.5478, max=0.5637, rel_change=2.86%
            var_time            : mean=0.0078, std=0.0000, min=0.0078, max=0.0078, rel_change=0.16%
            avg_compute_time    : mean=0.5581, std=0.0024, min=0.5557, max=0.5605, rel_change=0.86%
            median_compute_time : mean=0.5398, std=0.0026, min=0.5372, max=0.5423, rel_change=0.95%
            var_compute_time    : mean=0.0075, std=0.0000, min=0.0075, max=0.0075, rel_change=0.13%
            avg_draw_time       : mean=0.0163, std=0.0055, min=0.0108, max=0.0218, rel_change=66.90%
            median_draw_time    : mean=0.0160, std=0.0053, min=0.0106, max=0.0213, rel_change=66.86%
            var_draw_time       : mean=0.0000, std=0.0000, min=0.0000, max=0.0000, rel_change=67.83%

Overall trend in avg_time: decreasing (-1.5690e-02 change per experiment)
Conclusion: Performance changes slightly across experiments.
```

<br><br>

Experiment 3: Reflexion Order

```text
Number of experiments: 6

            avg_time            : mean=0.9300, std=0.7354, min=0.3231, max=2.4409, rel_change=227.71%
            median_time         : mean=0.8502, std=0.6128, min=0.3253, max=2.0917, rel_change=207.76%
            var_time            : mean=0.4635, std=0.9329, min=0.0002, max=2.5421, rel_change=548.45%
            avg_compute_time    : mean=0.9014, std=0.7194, min=0.3088, max=2.3798, rel_change=229.74%
            median_compute_time : mean=0.8241, std=0.6002, min=0.3108, max=2.0401, rel_change=209.83%
            var_compute_time    : mean=0.4449, std=0.8953, min=0.0002, max=2.4396, rel_change=548.32%
            avg_draw_time       : mean=0.0286, std=0.0160, min=0.0143, max=0.0611, rel_change=163.74%
            median_draw_time    : mean=0.0266, std=0.0129, min=0.0144, max=0.0521, rel_change=142.21%
            var_draw_time       : mean=0.0002, std=0.0004, min=0.0000, max=0.0011, rel_change=550.06%

Overall trend in avg_time: increasing (3.7365e-01 change per experiment)
Conclusion: Performance changes significantly across experiments.
```

<br><br>

Experiment 4: Detail Draw

```text
Number of experiments: 2

            avg_time            : mean=0.6282, std=0.0515, min=0.5767, max=0.6796, rel_change=16.39%
            median_time         : mean=0.6101, std=0.0507, min=0.5593, max=0.6608, rel_change=16.63%
            var_time            : mean=0.0115, std=0.0040, min=0.0074, max=0.0155, rel_change=70.54%
            avg_compute_time    : mean=0.5572, std=0.0020, min=0.5552, max=0.5592, rel_change=0.70%
            median_compute_time : mean=0.5430, std=0.0047, min=0.5383, max=0.5478, rel_change=1.75%
            var_compute_time    : mean=0.0074, std=0.0002, min=0.0072, max=0.0076, rel_change=5.32%
            avg_draw_time       : mean=0.0709, std=0.0495, min=0.0214, max=0.1204, rel_change=139.55%
            median_draw_time    : mean=0.0663, std=0.0454, min=0.0209, max=0.1117, rel_change=137.05%
            var_draw_time       : mean=0.0011, std=0.0011, min=0.0000, max=0.0022, rel_change=199.31%

Overall trend in avg_time: increasing (1.0292e-01 change per experiment)
Conclusion: Performance changes slightly across experiments.
```

<br><br>

Summary:

The Stepsize/amount of rays have the biggest impact on the performance. The other parameters have rather a small impact.

In [28]:
# cleanup
import os
for img in imgs:
    os.remove(img)